#### -- [INIT] -- 
***

##### **Import Libraries and Init Globals**
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [1]:
import pygame
from typing import NamedTuple, Tuple, List, Type, Callable
import os
import sys

class SystemContext(NamedTuple):
    screen :pygame.Surface
    clock :pygame.time.Clock

# System Context will be init on awake
class RenderContext(NamedTuple):
    WIDTH :int
    HEIGHT :int
    BACKGROUND_COLOR :tuple
    FPS :int

class AssetContext(NamedTuple):
    ASSET_FOLDER_PATH :str

# Statically define CONTEXT CONSTANTS
RENDER_CONTEXT = RenderContext(
    640, # RESOLUTION_WIDTH
    480, # RESOLUTION_HEIGHT
    (255, 255, 0), # BACKGROUND_COLOR
    30 # FPS
)

ASSET_CONTEXT = AssetContext(
    ASSET_FOLDER_PATH = os.path.join('assets','images')
)

def on_awake()-> SystemContext:
    """
    Initialize pygame
    - SYS_SCREEN
    - SYS_CLOCK
    """
    # Initialize pygame
    pygame.init()
    # Initialize the screen
    screen = pygame.display.set_mode(
        # @audit-ok 💨 : Use `HWSURFACE` and `DOUBLEBUF` for better performance
        (RENDER_CONTEXT.WIDTH, RENDER_CONTEXT.HEIGHT),
        pygame.HWSURFACE | pygame.DOUBLEBUF)
    # Initialize the clock
    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_ECS_Edition')
    icon = pygame.image.load(
        os.path.join(ASSET_CONTEXT.ASSET_FOLDER_PATH, 'app_icon.jpg')
    ).convert_alpha()
    pygame.display.set_icon(icon)

    return SystemContext(screen, clock)

def on_exit() -> None :
    """Handle exit event"""
    pygame.quit()
    sys.exit()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### -- [ENTITY] --
***
- generate a `Unique ID` for each `entity`

In [2]:
import uuid

class Entity:
    def __init__(self, id=None):
        """Simple Entity class with a unique identifier for each entity"""
        self.id = id or self.generate_unique_id()

    @staticmethod
    def generate_unique_id():
        return str(uuid.uuid4())

#### -- [COMPONENT] --
***

##### **Component Base**
- `Display`     : Position, Shape
- `Interaction` : Collider, Health

In [3]:
# Create component classes
# These classes should ONLY contain data, NO METHODS

# input handler
class Input:
    def __init__(self):
        pass

# transforms
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Rotation:
    def __init__(self, angle):
        self.angle = angle

class Velocity:
    def __init__(self, vx, vy):
        self.vx = vx
        self.vy = vy

# state
class Health:
    def __init__(self, health):
        self.health = health

# collision volume
class Collider:
    def __init__(self, radius):
        self.radius = radius

# graphics
class Shape:
    def __init__(self, points:List[Tuple[float,float]]):
        self.points = points

##### **Component Manager**
- `add`
- `remove`
- `get`
- `return entities` given `component set`

In [4]:
from abc import ABC, abstractmethod
from collections import defaultdict
from typing import TypeVar, Type, List, Generic

# `TypeVar` is a class from the `typing` module. It is used to create a generic  
# type variable `T`. This allows you to define a function that works with any 
# type, while still providing type hints for better code readability and 
# editor support.
T = TypeVar('T')

# Pythonic to name suffix ABC for abstract base classes, instead of `Interface``
class ComponentManagerABC(ABC):
    '''Interface for a Component Manager Class'''
    @abstractmethod
    def add_component(self, entity, component) -> None:
        """Add a component to an entity"""

    @abstractmethod
    def remove_component(self, entity, component_type: Type) -> None:
        """Remove a component of a specific type from an entity"""

    #  The `-> T` part indicates that the function returns a value of type `T`
    # - Since `T` is a generic type variable, it means that the return type of 
    # this function is the same as the type passed to the `component_type` 
    # parameter
    @abstractmethod
    def get_component(self, entity, component_type: Type[T]) -> T:
        """Get a component of a specific type from an entity"""

    @abstractmethod
    def get_entities_with_component(self, component_type: Type) -> List[Entity]:
        """Get a list of entities that have A SINGLE SPECIFIC COMPONENT type"""

    @abstractmethod
    def get_entities_with_all_components(self, component_types: List[Type]) -> List[Entity]:
        """Get a list of entities that have ALL of the specified COMPONENTS"""


# Implement a simple ComponentManager class that manages a dictionary of components
class ComponentManager(ComponentManagerABC):
    def __init__(self):
        # With a `defaultdict`, you can directly add a component without first 
        # checking if the `entity.id` exists in `self.components`. 
        # If the `entity.id` is not present, a new dictionary will be created 
        # automatically :
        # - simplifies the code for `add_component`
        self.components = defaultdict(dict)

    def add_component(self, entity, component) -> None:
        # - defaultdict will automatically create a new dictionary for the entity
        # if it does not exist : greatly simplifies the code
        # - check if component already exists
        # - When checking if an object is an instance of a class, it's recommended 
        # to use the `isinstance` function instead of directly comparing types
        # This allows you to handle cases where `component` is an instance of a 
        # subclass of the expected component type
        if any(isinstance(c, type(component)) for c in self.components[entity.id].values()):
            raise ValueError(f'[ERROR] tried to [add_component] but Component of type {type(component)} already exists on entity {entity.id}')
        self.components[entity.id][type(component)] = component

    def remove_component(self, entity, component_type:Type) -> None:
        try:
            del self.components[entity.id][component_type]
        except KeyError:
            raise ValueError(f'[ERROR] - tryied to [remove_component] - Component of type {component_type} does not exist on entity {entity.id}')

    def get_component(self, entity, component_type:Type[T]) -> T:
        # Because `None` is not compatible with the expected return type `T`
        # - To fix this issue, you can use the `in` keyword to check if the  
        # `component_type` key exists in the `self.components[entity.id]`  
        # dictionary before returning its value
        if entity.id in self.components and component_type in self.components[entity.id]:
            return self.components[entity.id][component_type]
        else:
            raise ValueError(f'[ERROR] - tryied to [get_component] - Component of type {component_type} does not exist on entity {entity.id}')
    
    def get_entities_with_component(self, component_type:Type) -> List[Entity]:
        entities = []
        for entity_id, components in self.components.items():
            if component_type in components:    
                entities.append(Entity(entity_id))
        return entities

    def get_entities_with_all_components(self, component_types:List[Type]) -> List[Entity]:
        entities = []
        # @audit 💨 : Convert the list of component types to a set for faster lookup
        # - in O(1) time instead of O(n) time where n is # of component types
        # - order of component types passed in does not matter, any permutation  
        # will give the same result
        component_set = set(component_types)
        for entity_id, components in self.components.items():
            if component_set.issubset(set(components.keys())):
                entities.append(Entity(entity_id))
        return entities


#### -- [ASSETS] --

##### **Generate**
- List of x,y `points`

In [5]:
import math
from numpy import random

def generate_circle(radius:float, num_points:int, func:Callable[[float], float]=lambda x:x)-> List[Tuple[float,float]]:
    """return a list of points that form a circle"""
    points = []
    angle = 2 * math.pi/num_points

    for i in range(num_points):
        # length = random.randint(int(radius * 0.5), radius)
        length = func(radius)
        x = length * math.cos(angle * i)
        y = length * math.sin(angle * i)
        points.append((x,y))

    return points

def generate_asteroid(radius, num_points) -> List[Tuple[float,float]]:
    """return a list of points that form an asteroid by adding noise to radius"""
    # `numpy.random.uniform` function to generate noise with a more uniform 
    # distribution between the minimum and maximum radii
    noise_lambda = lambda x: random.uniform(x*0.5, x)
    points = generate_circle(radius, num_points, noise_lambda)
    return points

def generate_ship(radius) -> List[Tuple[float,float]]:
    """return a list of points that form a ship"""
    points = generate_circle(radius, 3)
    return points

##### **SPAWN**

In [6]:
SHIP_HEALTH = 1
SHIP_SIZE = 30
ASTEROID_HEALTH = 100
ASTEROID_SIZE = 100
ASTEROID_VERTICES = 15
ASTEROID_MAX_SPEED = 5

def spawn_ship(component_manager: ComponentManager) -> None:
    'spawn a ship at the center of the screen'
    ship = Entity()
    ship_position = Position(
        int(RENDER_CONTEXT.WIDTH * 0.5), 
        int(RENDER_CONTEXT.HEIGHT * 0.5)
    )
    ship_input = Input()
    ship_health = Health(SHIP_HEALTH)
    ship_points = generate_ship(SHIP_SIZE)
    ship_shape = Shape(ship_points)

    component_manager.add_component(ship, ship_input)
    component_manager.add_component(ship, ship_position)
    component_manager.add_component(ship, ship_health)
    component_manager.add_component(ship, ship_shape)

def spawn_asteroid(component_manager: ComponentManager) -> None:
    'spawn an asteroid at a random positions'
    asteroid = Entity()
    asteroid_position = Position(
        random.randint(0, RENDER_CONTEXT.WIDTH), 
        random.randint(0,RENDER_CONTEXT.HEIGHT)
    )
    asteroid_velocity = Velocity(
        random.randint(-ASTEROID_MAX_SPEED, ASTEROID_MAX_SPEED),
        random.randint(-ASTEROID_MAX_SPEED, ASTEROID_MAX_SPEED)
    )
    asteroid_rotation = Rotation(
        random.randint(0, 359)
    )
    asteroid_health = Health(ASTEROID_HEALTH)
    asteroid_points = generate_asteroid(
        random.randint(int(ASTEROID_SIZE * 0.5), ASTEROID_SIZE), 
        ASTEROID_VERTICES
    )
    asteroid_shape = Shape(asteroid_points)

    component_manager.add_component(asteroid, asteroid_position)
    component_manager.add_component(asteroid, asteroid_velocity)
    component_manager.add_component(asteroid, asteroid_rotation)
    component_manager.add_component(asteroid, asteroid_health)
    component_manager.add_component(asteroid, asteroid_shape)

def spawn_asteroids(num_asteroids, component_manager:ComponentManager) -> None:
    'spawn a number of asteroids at random positions'
    for i in range(num_asteroids):
        spawn_asteroid(component_manager)

#### -- [SYSTEM]--
***
- Movement
- Render


In [12]:
from numpy import random

class SystemABC(ABC):
    @abstractmethod
    def update(self, cm:ComponentManager):
        """Update the system with the given component manager"""

# Implement Systems that operate on the components
class InputSystem(SystemABC):
    def update(self, cm :ComponentManager):
        """Hmmm... empty stub for SystemABC"""
        pass

    def update_with_event(self, cm: ComponentManager, event: pygame.event.Event):
        """Update the input component of entities with input"""
        entities = cm.get_entities_with_all_components([Input])
        for entity in entities:
            print(f'Handling event {event} for entity {entity}')

class MovementSystem(SystemABC):
    def update(self, cm :ComponentManager):
        """ 
        Update the position of entities with a velocity component 
        - with screen wrapping
        """
        entities = cm.get_entities_with_all_components([Position, Velocity])
        for entity in entities:
            velocity = cm.get_component(entity, Velocity)
            position = cm.get_component(entity, Position)
            position.x = (position.x + velocity.vx) % RENDER_CONTEXT.WIDTH
            position.y = (position.y + velocity.vy) % RENDER_CONTEXT.HEIGHT

class RenderSystem(SystemABC):
    def __init__(self, screen :pygame.Surface, clock :pygame.time.Clock):
        self.screen = screen
        self.clock = clock

    def _fill_background(self):
        rgb_random = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # self.screen.fill(RENDER_CONTEXT.BACKGROUND_COLOR)
        self.screen.fill(rgb_random)

    def update(self, cm :ComponentManager):
        self._fill_background()

        # get entities with POSITION (where) AND SHAPE (what) to render
        entities = cm.get_entities_with_all_components([Shape, Position])

        # Render all entities with a position and points component
        for entity in entities:
            position = cm.get_component(entity, Position)
            x,y = position.x, position.y
            shape = cm.get_component(entity, Shape)
            pygame.draw.polygon(
            self.screen, 
            (255, 255, 255), 
            [(x+px, y+py) for px,py in shape.points], 
            1
        )

        # Update the display
        pygame.display.update()
        # Limit the frame rate
        self.clock.tick(RENDER_CONTEXT.FPS)


#### -- [RUNTIME] --
***

In [14]:
# Initialization
context_sys = on_awake()
component_manager = ComponentManager()
input_system = InputSystem()
movement_system = MovementSystem()
render_system = RenderSystem(context_sys.screen, context_sys.clock)

# Create ship
spawn_ship(component_manager)
# Create asteroid
spawn_asteroids(5, component_manager)

# Main game loop
while True:
    try:
        # Set the frame rate
        context_sys.clock.tick(RENDER_CONTEXT.FPS)
        # Event Handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                on_exit()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_ESCAPE:
                    on_exit()
            # Update inputs
            input_system.update_with_event(component_manager, event)
        # Update positions
        movement_system.update(component_manager)
        # Update the display
        render_system.update(component_manager)
            
    except SystemExit:
        break

Handling event <Event(32768-ActiveEvent {'gain': 1, 'state': 2})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(32785-WindowFocusGained {'window': None})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(32768-ActiveEvent {'gain': 1, 'state': 1})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(32783-WindowEnter {'window': None})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(1024-MouseMotion {'pos': (302, 182), 'rel': (-220, 20), 'buttons': (0, 0, 0), 'touch': False, 'window': None})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(1024-MouseMotion {'pos': (301, 182), 'rel': (-1, 0), 'buttons': (0, 0, 0), 'touch': False, 'window': None})> for entity <__main__.Entity object at 0x158c693a0>
Handling event <Event(1024-MouseMotion {'pos': (300, 182), 'rel': (-1, 0), 'buttons': (0, 0, 0), 'touch': False, 'window': None})> for entity <__main__.Entity object at 0x15

#### -- [FOOTNOTES] --
***

Explain **`type checking`** in python
***

Some key points:

1. Type hints only `optionally enforce types` - you need a separate type  
checking tool. But they `clearly document the expected types` for your function.

2. You can specify types for all function arguments and the return value.

3. Use List[type] to denote a list of elements of type type.

4. Use Tuple[type1, type2] to denote a tuple with two elements of types type1 and type2.

5. You can use int, str, float, bool, etc. to specify basic types. And List,  
Tuple, Dict to specify compound types.

6. Type hints `do not affect runtime behavior` - they only provide `static`  
type checks.

7. You can use a tool like `mypy` to actually `run the type checks and enforce`  
the hints. 

8. Specifying strict type hints makes your code more robust, documented and  
catch bugs early. 

You can use type checking to ensure that a function only accepts a list of  
points (x, y coordinates). You can do it like this:

```python
from typing import List, Tuple

def function_name(points: List[Tuple[int, int]]) -> None:
  """Only accepts a list of (x, y) point tuples"""
  # Function logic here 
```

This uses Type Hints to specify that the points argument must be a List of  
Tuples, where each Tuple has two int elements. 
So these would be valid calls:

```python
function_name([(1, 2), (3, 4)]) 
function_name([[1, 2], [3, 4]])  # Also ok, treats each sublist as a point
```

But these would throw a TypeError:

```python
function_name((1, 2, 3))  # Not a list 
function_name([(1, '2')])  # Not ints 
function_name(['string', 'list'])  # Not tuples
```

Explain **`type checking`** in python as if I am 5
***

Imagine you have a toy box with different sections for different toys:  

- cars
- dolls 
- and building blocks...etc 

Your mom tells you to put the toys into the right sections to keep the toy box  
organized.

In Python, the toys are like "values" or "data," and the sections in the toy box  
are like "types" (like numbers, words, or lists). Type checking is like making  
sure you put the toys into the correct sections.

When you write a program in Python, you use different types of data, like  
numbers, words, or lists. Type checking helps make sure you use the right type  
of data in your program.

For example, if you want to add two numbers together, you need to make sure they  
are both numbers, not a number and a word. Type checking helps you do that.

Python has two ways to do type checking:  

1. **Dynamic type checking**: Python automatically checks the types of the data  
when you run the program, just like your mom checks the toys in the box when  
you're done playing. If something is wrong, Python will tell you with an error  
message.

2. **Static type checking**: You can add "hints" to your program to tell Python  
what type of data you expect to use, just like putting labels on the toy box  
sections. This way, a tool like `mypy` or an editor like Visual Studio Code can  
check the types before you run the program and help you find any mistakes.

Type checking makes your program safer and easier to understand, just like  
keeping your toy box organized makes it easier to find and play with your toys.

Explain python **`set operations`** as if I’m 5
***

Imagine you have a box of different toy shapes:  
- triangles
- circles
- squares  

Now, let's say you have another box with shapes like circles, squares, and **`stars`**  
In Python, these boxes are like sets, and the shapes inside them are like the  
elements of the sets.

Now, let's learn some Python set operations using these toy shapes:

1. **Union (|)**: If you take all the shapes from both boxes and put them  
**`together`**, you will have a new box with all the shapes: 
- triangles 
- circles 
- squares 
- and **`stars`** 

This new box represents the union of the two sets.  

   ```python
   box1 = {"triangle", "circle", "square"}
   box2 = {"circle", "square", "star"}
   all_shapes = box1 | box2  # {"triangle", "circle", "square", "star"}
   ```

2. **Intersection (&)**: If you look for shapes that are in both box1 and box2,  
you will find circles and squares. This is the intersection of the two sets.

   ```python
   common_shapes = box1 & box2  # {"circle", "square"}
   ```

3. **Difference (-)**: If you want to find the shapes that are only in box1  
(and not in box2), you will have triangles. This is the difference between  
the two sets.

   ```python
   unique_shapes_box1 = box1 - box2  # {"triangle"}
   ```

4. **Symmetric Difference (^)**: If you want to find the shapes that are unique  
to each box (not in both), you will have triangles and stars. This is the  
symmetric difference between the two sets.

   ```python
   unique_shapes_both = box1 ^ box2  # {"triangle", "star"}
   ```

5. **issubset()**: If you want to check if all shapes in one box are also in  
another box, you can use `issubset()`. For example, if you have a small box with  
only circles and squares, you can check if all the shapes in the small box are  
also in box1.

   ```python
   small_box = {"circle", "square"}
   is_small_box_in_box1 = small_box.issubset(box1)  # True
   ```

These are the basic Python set operations explained using toy shapes. Just like  
you can play with toy shapes in the boxes, you can use set operations in Python  
to work with sets of elements.

Code review this **`naive`** implementation and explain refactoring options
***    

```python
def get_entities_with_all_components(self, component_types:List[Type]) -> List[Entity]:
    entities = []
    for entity_id, components in self.components.items():
        has_all_components = True
        for type in component_types:
            if type not in components:
                has_all_components = False
                break
        if has_all_components:
            entities.append(Entity(entity_id))
    return entities
```

`get_entities_with_all_components` - `GPT4` suggestion
***

Here's a reviewed and refactored version of your `get_entities_with_all_components`  
method:

```python
from typing import List, Type

def get_entities_with_all_components(self, component_types: List[Type]) -> List[Entity]:
    entities = []
    for entity_id, components in self.components.items():
        if all(component_type in components for component_type in component_types):
            entities.append(Entity(entity_id))
    return entities
```

Here are the changes I made:

1. Removed the `has_all_components` flag and used the `all()` built-in function  
instead. This function returns `True` if all elements of the iterable are true  
(or if the iterable is empty). By using a list comprehension inside `all()`,  
we can check if all the required component types are present in the `components`  
dictionary for the given entity. This simplifies the code and makes it more  
Pythonic.

`get_entities_with_all_components` - `Claude` suggestion
***

```python
def get_entities_with_all_components(self, component_types:List[Type]) -> List[Entity]:
    entities = []
    # @audit 💨 : Convert the list of component types to a set for faster lookup
    # - in O(1) time instead of O(n) time where n is # of component types
    # - order of component types passed in does not matter, any permutation  
    # will give the same result
    component_set = set(component_types)
    for entity_id, components in self.components.items():
        if component_set.issubset(set(components.keys())):
            entities.append(Entity(entity_id))
    return entities
```

**GPT4 comments**

This refactored version of the `get_entities_with_all_components` method  
introduces a few changes:

1. It converts the `component_types` list to a `component_set` set for faster  
lookup. Sets offer O(1) average time complexity for testing membership compared  
to lists, which have O(n) time complexity where n is the number of component  
types.

2. It uses the `issubset()` method to check if all component types are present  
in the `components` dictionary for the given entity. This method checks if every  
element in the `component_set` is in the specified iterable (in this case, a set  
of the component keys).

**Tradeoffs:**

1. **Performance**: The refactored version should have better performance in  
cases where the number of component types is large. This is because set  
membership tests are faster than list membership tests.

2. **Memory**: The refactored version uses a little more memory since it creates  
a set from the input list and another set from the component keys. This memory  
overhead should be negligible in most cases.

3. **Readability**: The refactored version is more concise and takes advantage  
of Python's built-in set operations, making it more Pythonic. However, it might  
be slightly less intuitive for developers who are not familiar with set  
operations.

**Functional differences:**

There are no significant functional differences between the two implementations.  
Both versions return a list of entities that have all the specified component  
types. The order of the component types passed to the function does not affect  
the result in either version.

Overall, the refactored version is a good improvement, especially if the `number`  
of `component types becomes large`. The performance gain from using sets for  
membership tests and the concise code using set operations make this version  
preferable in most cases.
